### Prérequis

Pour exécuter le code si joint, il faut installer la librairie jupylet

Voici le guide d'installation

[Guide d'installation](https://jupylet.readthedocs.io/en/latest/programmers_reference_guide/getting_started.html)


```pip install jupilet```

Ensuite il faut procéder à la post-installation.

```python -m jupylet postinstall```

### Pong

Exécutez ce programme et regarder ce qui se passe.

Cliquer la souris dans le l'image du jeu.  Pour bouger les pallettes il faut utiliser les touches flèche gauche et flèche droite pour une palette ainsi que la touche A et D pour l'autre palette


* le son est fournis par  [freesound](https://freesound.org/people/NoiseCollector/sounds/4359/).
* Les fonts de Commodore 64 sont fournis par  [KreativeKorp](https://www.kreativekorp.com/software/fonts/c64.shtml).

Ce jeu est fournis par le site web https://jupylet.readthedocs.io/en/latest/index.html




In [ ]:
import logging
import sys
import os

import numpy as np

In [ ]:
sys.path.insert(0, os.path.abspath('./..'))

Ici on ajoute les librairies de code pour nous permettre d'avoir des fonctionnalités tel que la capacité d'écrire sur l'écran (jupylet.app), d'écrire du texte (jupylet.label), d'afficher des petits morceaux d'image (jupylet.sprite), d'ajouter du son à notre jeu (jupylet.audio.sample)

In [ ]:
import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

In [ ]:
app = App()#log_level=logging.INFO)

Ici on défini les couleurs de notre fond d'écran et de l'avant plan ..

In [ ]:
background = '#3e32a2'
foreground = '#7c71da'

Ici on affecte une matrice de 32 * 32 de chiffre 1.  

![](.\images\matrix-example2.png)

Par contre la matrice a0 est de grandeur de 32 ligne par 32 colonnes, ce serait assez long ajouter toutes les colonnes et les lignes dans le dessin.

Ensuite on multiplie la matrice par 255, et voilà ce que ça donnera ...

![](.\images\matrix-example3.png)

Encore là, il faut compter 32 lignes et 32 colonnes pour la variable a0

Pour la variable a1, on aurra aussi la valeur 255 dans chacune des cases, cette matrice aura 128 lignes et 16 colonnes.

Enfin la variables a2 est un peu plus compliquée, 

Au lieu d'avoir une matrice, 2 par 2, on aura un cube comme un cube Rubix.

Les trois axes (X, Y, Z) auront différentes valeurs.  
Par example l'axe X sera relié à la largeur 
L'axe Y sera relié à la hauteur de l'application 
L'axe Z représente une profondeur, dans notre cas, c'est pour accumuler la couleur à une position précise.

![](.\images\cube.png)

L'image représente un cube de 5 par 5 par 5.  Dans notre case c'est Hauteur de l'application par Largeur de l'application par 3.


In [ ]:
a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255

A cet endroit on définit des image en leur donnant la grandeur qu'on les veut.

Par exemple, **ball** sera la balle qui se déplace.
padl et padr seront les palettes qui se déplacent pour intercepter la balle.

Nous donnons les coordonnées selons les tableaux que nous avons fait préalablement pour indiquer à nos images (Sprite) les positions disponibles dans le notre jeu, ça nous permettra de déplacer les images en vertical (X) et horizontal (y)

field est notre terrain de jeu, l'arrière plan du jeu.

Pour définir l'image, on indique la position dans la variable **x** et **y** comme par exemple, les pallettes **padl** et **palr** sont situés soit près à gauche **48** ou à la droite (**app.width - 48**)

![](.\images\paddle.png)


Pour ce qui est de l'arrière plan du jeu, revenons à l'instruction précédente 
**a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255**

Nous indiquons dans le tableau que nous voulons remplir 90% du tableau en hauteur et en largeur, la troisième dimension du tableau permet de conserver la couleur.

Ceci donnera l'effet d'avoir les le tour du canvas (10%) avec la couleur de fond. Pour celui-ci on le centre en **x** et **y** dans notre canevas. 

In [ ]:
ball = Sprite(a0, y=app.height/2, x=app.width/2)

padl = Sprite(a1, y=app.height/2, x=48)
padr = Sprite(a1, y=app.height/2, x=app.width-48)

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background) 

Comme tout bon jeu, on doit ajouter du son, et pour ça, on lit un fichier de son.  Comme lorsque nous allons dans une application pour écouter de la musique nous choisissons un fichier qui nous fait jouer de la musique.

In [ ]:
pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()

Tout bon jeu à aussi un résultat, il faut l'afficher, donc on utilise un **Label** qui nous permet de visualiser du texte dans notre jeu.  On lui indique des paramètres comme la taille, la position dans notre jeu, le type de lettre.
Le type de lettre est nommé font:
Voici une liste de fonts déjà existants qu'on peut utiliser pour écrire notre texte dans notre jeu.

![](.\images\font.png)

In [ ]:
scorel = Label(
    '0', font_size=42, color=foreground, 
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground, 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)

Ensuite, il faut bien afficher nos images et notre texte, on utilise la commande **draw** de chacun des textes et des images pour que ceux-ci puissent s'afficher. 

Le premier **field** est le plan de jeu
**scorel** et **scorer** sont les résultats de chacun des joueurs (celui à gauche et celui à droite)

Ensuite on dessine la balle **ball.draw()**

Enfin on dessine les 2 palettes **padl** et **padr**

In [ ]:
@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()

C'est bien d'afficher nos palettes et notre balle mais si elles ne bouge pas alors notre jeu sera très ennuyant.  C'est pour ça qu'on doit ajouter du mouvement et connaître la position de notre balle et de nos pallettes à tout moment pour savoir comment réagir au déplacement de la balle. Ainsi que les touches appuyées par l'utilisateur.

Par exemple lorsque le joueur appuie sur la flèche gauche (left) notre variable d'état **left** sera à vrai (true) et la même chose pour la variable **right** sera à vraie lorsque sa touche sera appuyé.

Les touches A et D seront aussi vérifiés.

La variable **pyl** permet de regarder la position de la palette à gauche dans la partie haute de la palette, la variable nous renseignera si la palette est trop haute ou trop basse, c'est bien sûr, on ne veut pas que la passe sorte du jeu, la palette doit rester dans le cadre du jeu.

La variable **pyr** elle aussi conserve la position du haut mais cette fois de la palette a droite.

**pyl** et **pyr** sont assigné au milieu de l'écran pour le début.


La variable **ayl** calcul l'espace à gauche que la palette gauche protège, la même chose pour la variable **ayr** à droite.




In [ ]:
state = State(
    
    sl = 0,
    sr = 0,
    
    bvx = 192,
    bvy = 192,
    
    vyl = 0,
    pyl = app.height/2,

    vyr = 0,
    pyr = app.height/2,

    left = False,
    right = False,

    key_a = False,
    key_d = False,
)

In [ ]:
@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:
        
        if key == keys.LEFT:
            state.left = True

        if key == keys.RIGHT:
            state.right = True

        if key == keys.A:
            state.key_a = True

        if key == keys.D:
            state.key_d = True

    if action == keys.ACTION_RELEASE:

    
        if key == keys.LEFT:
            state.left = False

        if key == keys.RIGHT:
            state.right = False

        if key == keys.A:
            state.key_a = False

        if key == keys.D:
            state.key_d = False

Cette fonction est appelée à chaque 120ième de seconde.  Donc c'est comme une horloge qui appelle la fonction.  L'objectif de la fonction est d'évaluer la position des deux palettes à des moments précis pour les dessigner aux endroits appropriés.

Les premières instructions s'assurent que les 2 palettes restent dans le cadre du jeu.


    if state.right:
        state.pyr = min(app.height, state.pyr + dt * 512)
        
    if state.left:
        state.pyr = max(0, state.pyr - dt * 512)
        
    if state.key_a:
        state.pyl = min(app.height, state.pyl + dt * 512)
        
    if state.key_d:
        state.pyl = max(0, state.pyl - dt * 512)

    On fait un calcul de déplacement entre la distance du parcourue entre le dernier dessin et la position au temps précis, on peut ajouter un certain facteur car les valeurs numérique sont très petites. Dans ce cas nous avons mis 100, mais on aurait pu mettre une autre valeur plus grande ou plus petite.      
        

    Ici on calcul l'accélération des palettes.  Si une touche n'était pas appuyée alors l'accélération sera constante, on ajoute une constante à l'équation (dans ce cas 5) car la distance est très petite et plus qu'on augmente le nombre plusque l'accélération sera grande.        
    ayl = 5 * (state.pyl - padl.y)
    ayr = 5 * (state.pyr - padr.y)


    Ici nous calculons la vitesse de la palette, à chaque fois que la fonction est appelée (1/120) seconde, la vitesse  (0.9), donc on obtiendra 90% de la vitesse initiale.

    L'utilisation de la variable **dt** nous permet de faire le rapport entre le temps passer depuis le dernier dessin, donc si nous passons plus de temps entre les intervalles, nous nous déplacerons plus.  En fait le fait de multiplier par **dt** nous permet d'avoir la même vitesse de déplacement peut importe les intervalles de temps.

    Le but ici est de calculer la vitesse que nous sommes rendu actuellement, si la variable **state.pyl** à augmenter car une clé était appuyée alors l'accélération sera plus grande donc par le fait même la vitesse sera aussi augmentée, elle ne sera pas constante.


    Nous calculons le nouveau composant de vitesse **(ayl * dt)** et nous l'ajoutons à la vitesse que nous avons déjà
    
    Voici la formule de la vitesse : 
    v = v0 + a*t

    Dans notre cas v0 est **state.vyl**, elle avait été calculée le cycle précédent, on peut ajouter la nouvelle composante de vitesse.
    Par contre, on sait très bien que la vitesse n'est pas stable dans la nature, il y a la gravité qui la retient.  Pour aider un peut le jeu on diminue la vitesse précédente à 90% de la vitesse initiale et on ajoute le nouveau composant de vitesse.  Ceci permettra de faire ralentir un petit peut à la fois la vitesse jusqu'à en arriver au point d'arrêt

    Vitesse   = Vitesse * 90%   + Nouvelle vitesse 

    state.vyl = state.vyl * 0.9 + (ayl * dt)
    state.vyr = state.vyr * 0.9 + (ayr * dt)


    Une fois qu'on a notre vitesse résultante, on peut calculer la position de la palette en donnant un nouveau centre à notre image (Sprite) qui est le déplacement de son dernier dessin avec la vitesse de ce cycle.

    Formule Distance = Vitesse * temps

    Distance de la palette est la position précédente + la distance faite dans ce cycle.

    padl.y += state.vyl * dt
    padr.y += state.vyr * dt
    

    A ce moment, on protège le fait de ne pas dessiner les palettes en dehors du cadre du jeu.

    padr.clip_position(app.width, app.height)
    padl.clip_position(app.width, app.height)   



In [ ]:
@app.run_me_every(1/60)
def update_pads(ct, dt):
        
    if state.right:
        state.pyr = min(app.height, state.pyr + dt * 512)
        
    if state.left:
        state.pyr = max(0, state.pyr - dt * 512)
        
    if state.key_a:
        state.pyl = min(app.height, state.pyl + dt * 512)
        
    if state.key_d:
        state.pyl = max(0, state.pyl - dt * 512)
        
    ayl = 10 * (state.pyl - padl.y)
    ayr = 10 * (state.pyr - padr.y)

    
    state.vyl = state.vyl * 0.9 + (ayl * dt)
    state.vyr = state.vyr * 0.9 + (ayr * dt)
    
    
    padl.y += state.vyl * dt
    padr.y += state.vyr * dt
    
    padr.clip_position(app.width, app.height)
    padl.clip_position(app.width, app.height)

Maintenant c'est le tour de la balle,  
Il faut déplacer la balle dans un intervale précis, encore une fois tous les 1 soixantième de secondes.


    Maintenant on calcule la vitesse totale en horizontal et vertical
    
    **bs0 = state.bvx ** 2 + state.bvy ** 2**
    
    On calcule une portion de la rotation de l'angle selon le temps (1/60) donc l'angle sera 200 par seconde.  

    ball.angle += 200 * dt

    On calcule le déplacement de la balle en horizontal selon la vitesse de la balle
    
    ball.x += state.bvx * dt

    On calcule le déplacement de la balle en vertical selon la vitesse de la balle
    ball.y += state.bvy * dt
    

    Si la balle atteint le haut du jeu, on doit la retourner dans le jeu en calculant l'angle de retour, on ramene la balle à la position maximum du jeu (pas à l'extérieur) on inverse sa velocité.


    if ball.top >= app.height:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.top - app.height
        state.bvy = -state.bvy
        
    Si la balle a atteint le bas du jeu, on la retourne comme ci-haut en inversant la vitesse verticale **bvy**

    if ball.bottom <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.bottom
        state.bvy = -state.bvy

    Si la balle dépasse à droite, on ramene la composant vertical de la balle (x) dans le jeu, ensuite il faut changer la vélocité de X et de Y car la balle va suivre un angle inverse    

    On va inversé la vélocité horizontale (x) car la balle passe de droite à gauche.  Pour la vélocité verticale, on garde la même direction.  Donc la vitesse choisie (192) conserve le même signe en verticale et change de signe en horizontal. 



    Enfin, il faut aussi donner un point à l'autre joueur, celui de gauche, si la balle se rend à droite complètement.

    if ball.right >= app.width:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.right - app.width
        
        state.bvx = -192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sl += 1
        scorel.text = str(state.sl)
        

    Pour la gauche, c'est pratiquement le même code, on inverse le signe en horizontale (bvx) et on garde le même signe en vertical (bvy), là-aussi le pointage augmente, mais cette fois-ci, pour l'autre joueur.

    if ball.left <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.left
        
        state.bvx = 192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sr += 1
        scorer.text = str(state.sr)

    Ici on va regarder si la palette est proche de la balle du côté droit du jeu.   Si c'est le cas, on repositionne la balle pour la coller sur la pallette, on inverse la vitesse horizontale de la balle **bvx** et on augmente la vitesse de la balle de 50% de la vitesse de la palette 

    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x -= ball.right - padr.left
            state.bvx = -state.bvx
            state.bvy += state.vyr / 2
            

    Maintenant on s'occupe du côté gauche du jeu et on fait sensiblement le même traitement pour la balle.  

    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x += ball.left - padl.right
            state.bvx = -state.bvx
            state.bvy += state.vyl / 2
            
    Maintenant on recalcule la vitesse totale en horizontal et vertical        
            
    bs1 = state.bvx ** 2 + state.bvy ** 2

    Ici on ajuste la vitesse de la balle horizontale si elle est trop basse par rapport au début de la fonction

    if bs1 < 0.9 * bs0:
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)

    ball.wrap_position(app.width, app.height)

In [ ]:
@app.run_me_every(1/60)
def update_ball(ct, dt):
    
    bs0 = state.bvx ** 2 + state.bvy ** 2
    
    ball.angle += 200 * dt
    
    ball.x += state.bvx * dt
    ball.y += state.bvy * dt
    
    if ball.top >= app.height:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.top - app.height
        state.bvy = -state.bvy
        
    if ball.bottom <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.y -= ball.bottom
        state.bvy = -state.bvy
        
    if ball.right >= app.width:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.right - app.width
        
        state.bvx = -192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sl += 1
        scorel.text = str(state.sl)
        
     # + ' ' + state.pyr + ' ' + padl.y    
    if ball.left <= 0:
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
        ball.x -= ball.left
        
        state.bvx = 192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sr += 1
        scorer.text = str(state.sr)
        
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x -= ball.right - padr.left
            state.bvx = -state.bvx
            state.bvy += state.vyr / 2
            
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)
            ball.x += ball.left - padl.right
            state.bvx = -state.bvx
            state.bvy += state.vyl / 2
            
    bs1 = state.bvx ** 2 + state.bvy ** 2
    
    if bs1 < 0.9 * bs0:
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)

    ball.wrap_position(app.width, app.height)

Maintenant, executons le programme

In [ ]:
app.run()